# Basic useage of the DSMS-Python-SDK

Before you run this tutorial: make sure to have access to an DSMS-instance of your interest, that you have installed this package and that you have copied the needed variables such as the `DSMS_HOST_URL` and `DSMS_TOKEN` into an `.env`-file.

First of all, make let us import the needed classes and functions for this tutortial.

In [1]:
import os
from pprint import pprint

from dotenv import load_dotenv

from dsms import DSMS, KItem

Now source the environmental variables from an `.env` file and start the DSMS-session.

In [2]:
#specify path to an arbitrary file
env = os.path.join("..", ".env")

# start the session
load_dotenv(env)

dsms = DSMS()

### 1: Introduction

We can see which kind of DSMS-object we own as a user:

In [3]:
dsms.kitems

[]

We can investigate what a KItem needs in order to be created. KItems are entirely based on [`Pydantic`](https://docs.pydantic.dev/latest/)-Models (v2), hence the properties (in `Pydantic` called `Fields`) are automatically validated once we set them. 

The schema of the KItem itself is a JSON schema which is machine-readable and can be directly incorporated into [Swagger](https://swagger.io/tools/swagger-ui/)-supported APIs like e.g. [`FastAPI`](https://fastapi.tiangolo.com/).

In [4]:
pprint(KItem.model_json_schema())

{'$defs': {'AdditionalProperties': {'description': 'Additional properties of',
                                    'properties': {'triggeredUponUpload': {'default': False,
                                                                           'description': 'Whether '
                                                                                          'the '
                                                                                          'app '
                                                                                          'should '
                                                                                          'be '
                                                                                          'triggered '
                                                                                          'when '
                                                                                          'a '
                                         

c:\Anaconda3\envs\sdk\Lib\site-packages\pydantic\json_schema.py:2099: PydanticJsonSchemaWarning: Default value <property object at 0x000001B8525FA2A0> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


We can investigate the KTypes defined in the remote instance:

In [5]:
for ktype in dsms.ktypes:
    print(ktype)

KTypes.Organization
KTypes.Expert
KTypes.App
KTypes.DatasetCatalog
KTypes.TestSeries
KTypes.TestMatthias


### 2: Create KItems

We can make new KItems by simple class-initiation:

In [6]:
item = KItem(
    name="foo123",
    ktype_id=dsms.ktypes.DatasetCatalog,
    custom_properties={"foo": "bar"},
)

item

KItem(

	name = foo123, 

	id = 7683a8c0-908e-4528-82ce-0b6b7140f7ab, 

	ktype_id = KTypes.DatasetCatalog, 

	slug = foo123, 

	annotations = [], 

	attachments = [], 

	linked_kitems = [], 

	affiliations = [], 

	authors = [], 

	avatar_exists = False, 

	contacts = [], 

	created_at = None, 

	updated_at = None, 

	external_links = [], 

	kitem_apps = [], 

	summary = None, 

	user_groups = [], 

	custom_properties = CustomProperties(content={'foo': 'bar'})
)

Remember: changes are only syncronized with the DSMS when you call the `commit`-method:

In [7]:
dsms.commit()

As we can see, the object we created before running the `commit`-method has automatically been updated, e.g. with the creation- and update-timestamp:

In [8]:
item

KItem(

	name = foo123, 

	id = 7683a8c0-908e-4528-82ce-0b6b7140f7ab, 

	ktype_id = KTypes.DatasetCatalog, 

	slug = foo123, 

	annotations = [], 

	attachments = [], 

	linked_kitems = [], 

	affiliations = [], 

	authors = [], 

	avatar_exists = False, 

	contacts = [], 

	created_at = None, 

	updated_at = None, 

	external_links = [], 

	kitem_apps = [], 

	summary = None, 

	user_groups = [], 

	custom_properties = CustomProperties(content={'foo': 'bar'})
)

And the list of our KItems in the DSMS has been updated as well:

In [9]:
dsms.kitems

[KItem(
 
 	name = foo123, 
 
 	id = 7683a8c0-908e-4528-82ce-0b6b7140f7ab, 
 
 	ktype_id = dataset-catalog, 
 
 	slug = foo123, 
 
 	annotations = [], 
 
 	attachments = [], 
 
 	linked_kitems = [], 
 
 	affiliations = [], 
 
 	authors = [
 		Author(user_id=4440f8c8-f443-4114-a7a7-d8e2a6b5d776)
 	], 
 
 	avatar_exists = False, 
 
 	contacts = [], 
 
 	created_at = 2024-02-27 15:40:43.372811, 
 
 	updated_at = 2024-02-27 15:40:43.372811, 
 
 	external_links = [], 
 
 	kitem_apps = [], 
 
 	summary = None, 
 
 	user_groups = [], 
 
 	custom_properties = CustomProperties(content={'foo': 'bar'})
 )]

### 3. Update KItems

Now, we would like to update the properties of our KItem we created previously.

Depending on the schema of each property (see `KItem.model_schema_json()` in the **Introduction** of this tutorial), we can simply use the standard `list`-method as we know them from basic Python (e.g. for the `annotations`, `attachments`, `external_link`, etc). 


Other properties which are not `list`-like can be simply set by attribute-assignment (e.g. `name`, `slug`, `ktype_id`, etc).

In [10]:
# specify the path to any arbitrary file to be uploaded
file = os.path.join("..", "README.md")

item.name = "foobar"
item.custom_properties.update({"foobar": "foobar"})
item.attachments.append({"name": file})
item.annotations.append(
    {
        "iri": "www.example.org/foo",
        "name": "example class",
        "namespace": "www.example.org",
    }
)
item.external_links.append(
    {"url": "http://example.org", "label": "example link"}
)
item.contacts.append({"name": "foo", "email": "foo@bar.mail"})
item.affiliations.append({"name": "foobar team"})
item.user_groups.append({"name": "foogroup", "group_id": "123"})

Changes are sent to the DSMS through the `commit`-method again.

In [11]:
dsms.commit()

We can see now that e.g. the local system path of the attachment is changed to a simply file name, which means that the upload was successful. If not so, an error would have beem thrown during the `commit`.

Furthermore we can also download the file we uploaded again:

In [12]:
for file in item.attachments:
    download = file.download()

    print("\t\t\t Downloaded file:", file)
    print("|------------------------------------Beginning of file------------------------------------|")
    print(download)
    print("|---------------------------------------End of file---------------------------------------|")

			 Downloaded file: Attachment(name=README.md)
|------------------------------------Beginning of file------------------------------------|
# DSMS-SDK
Python SDK core-package for interacting with the Dataspace Management System (DSMS)


## Authors

[Matthias Büschelberger](mailto:matthias.bueschelberger@iwm.fraunhofer.de) (Fraunhofer Institute for Mechanics of Materials IWM)

[Yoav Nahshon](mailto:yoav.nahshon@iwm.fraunhofer.de) (Fraunhofer Institute for Mechanics of Materials IWM)

[Pablo De Andres](mailto:pablo.de.andres@iwm.fraunhofer.de) (Fraunhofer Institute for Mechanics of Materials IWM)

## License

This project is licensed under the BSD 3-Clause. See the LICENSE file for more information.

## Usage

The SDK provides a general Python interface to a remote DSMS deployment, allowing users to access, store and link data in a DSMS instance easily and safely. The package provides the following main capabilities:

- Managing Knowledge-Items (KItems), which are data instances of an ex

### 4: Delete KItems and their properties

We can also remove properties from the KItem without deleting the KItem itself.

For the `list`-like properties, we can use the standard `list`-methods from basic Python again (e.g. `pop`, `remove`, etc. or the `del`-operator).

For the other, non-`list`-like properties, we can simply use the attribute-assignment again.

When we only want single parts of the properties in the KItem, we can do it like this:

In [13]:
item.attachments.pop(0)
item.annotations.pop(0)
item.external_links.pop(0)
item.contacts.pop(0)
item.user_groups.pop(0)

UserGroup(name=foogroup, group_id=123)

However, we can also reset the entire property by setting it to e.g. an empty list again:

In [14]:
item.affiliations = []

See the changes:

In [15]:
item

KItem(

	name = foobar, 

	id = 7683a8c0-908e-4528-82ce-0b6b7140f7ab, 

	ktype_id = KTypes.DatasetCatalog, 

	slug = foo123, 

	annotations = [], 

	attachments = [], 

	linked_kitems = [], 

	affiliations = [], 

	authors = [], 

	avatar_exists = False, 

	contacts = [], 

	created_at = None, 

	updated_at = None, 

	external_links = [], 

	kitem_apps = [], 

	summary = None, 

	user_groups = [], 

	custom_properties = CustomProperties(content={'foobar': 'foobar'})
)

Send the changes to the DSMS with the `commit`-method:

In [16]:
dsms.commit()

However, we can also delete the whole KItem from the DSMS by applying the `del`-operator to the `dsms`-object with the individual `KItem`-object:

In [17]:
del dsms[item]

Commit the changes:

In [18]:
dsms.commit()

As we can see know, our KItem has been removed from our inventory:

In [19]:
dsms.kitems

[]

### 5: Search for KItems

In the last unit of this tutorial, we would like to search for specfic KItems we created in the DSMS.

For this purpose, we will firstly create some KItems and apply the `search`-method on the `DSMS`-object later on in order to find them again in the DSMS.

We also wnat to demonstrate here, that we can link KItems to each other in order to find e.g. a related item of type `DatasetCatalog`. For this strategy, we are using the `linked_kitems`-attribute and the `id` of the item which we would like to link.

The procedure looks like this:

In [20]:
item = KItem(
    name="foo 1",
    ktype_id=dsms.ktypes.DatasetCatalog
)

item2 = KItem(
    name="foo 2",
    ktype_id=dsms.ktypes.Organization,
    linked_kitems=[item],
    annotations=[
        {
            "iri": "www.example.org/foo",
            "name": "foo",
            "namespace": "www.example.org",
        }
    ],
)
item3 = KItem(
    name="foo 3", 
    ktype_id=dsms.ktypes.Organization
)
item4 = KItem(
    name="foo 4",
    ktype_id=dsms.ktypes.Organization,
    annotations=[
        {
            "iri": "www.example.org/bar",
            "name": "bar",
            "namespace": "https://www.example.org",
        }
    ],
)

dsms.commit()

Now, we are apply to search for e.g. kitems of type `DatasetCatalog`:

In [21]:
dsms.search(ktypes=[dsms.ktypes.DatasetCatalog])

[KItem(
 
 	name = foo 1, 
 
 	id = d475776e-5649-4add-b5f9-08baf1d4f43b, 
 
 	ktype_id = dataset-catalog, 
 
 	slug = foo1, 
 
 	annotations = [], 
 
 	attachments = [], 
 
 	linked_kitems = [
 		LinkedKItem(id=7664eeae-b283-40ce-aa26-607b871ccd2a, source_id=d475776e-5649-4add-b5f9-08baf1d4f43b)
 	], 
 
 	affiliations = [], 
 
 	authors = [
 		Author(user_id=4440f8c8-f443-4114-a7a7-d8e2a6b5d776)
 	], 
 
 	avatar_exists = False, 
 
 	contacts = [], 
 
 	created_at = 2024-02-27 15:41:24.671471, 
 
 	updated_at = 2024-02-27 15:41:24.671471, 
 
 	external_links = [], 
 
 	kitem_apps = [], 
 
 	summary = None, 
 
 	user_groups = [], 
 
 	custom_properties = CustomProperties(content={})
 ),
 KItem(
 
 	name = material constants, 
 
 	id = ea0bb87b-3a26-48be-84e1-bec9b99a1204, 
 
 	ktype_id = dataset-catalog, 
 
 	slug = material_constants, 
 
 	annotations = [
 		Annotation(iri=https://w3id.org/steel/ProcessOntology/MaterialCard, name=MaterialCard, namespace=https://w3id.org/steel/ProcessOn

... and for all of type `Organization` and `DatasetCatalog`:

In [22]:
dsms.search(ktypes=[dsms.ktypes.Organization, dsms.ktypes.DatasetCatalog])

[KItem(
 
 	name = foo 1, 
 
 	id = d475776e-5649-4add-b5f9-08baf1d4f43b, 
 
 	ktype_id = dataset-catalog, 
 
 	slug = foo1, 
 
 	annotations = [], 
 
 	attachments = [], 
 
 	linked_kitems = [
 		LinkedKItem(id=7664eeae-b283-40ce-aa26-607b871ccd2a, source_id=d475776e-5649-4add-b5f9-08baf1d4f43b)
 	], 
 
 	affiliations = [], 
 
 	authors = [
 		Author(user_id=4440f8c8-f443-4114-a7a7-d8e2a6b5d776)
 	], 
 
 	avatar_exists = False, 
 
 	contacts = [], 
 
 	created_at = 2024-02-27 15:41:24.671471, 
 
 	updated_at = 2024-02-27 15:41:24.671471, 
 
 	external_links = [], 
 
 	kitem_apps = [], 
 
 	summary = None, 
 
 	user_groups = [], 
 
 	custom_properties = CustomProperties(content={})
 ),
 KItem(
 
 	name = foo 2, 
 
 	id = 7664eeae-b283-40ce-aa26-607b871ccd2a, 
 
 	ktype_id = organization, 
 
 	slug = foo2, 
 
 	annotations = [
 		Annotation(iri=www.example.org/foo, name=foo, namespace=www.example.org)
 	], 
 
 	attachments = [], 
 
 	linked_kitems = [
 		LinkedKItem(id=d475776e-5649-4ad

... or for all of type `DatasetCatalog` with `foo` in the name:

In [23]:
dsms.search(query="foo", ktypes=[dsms.ktypes.DatasetCatalog])

[KItem(
 
 	name = foo 1, 
 
 	id = d475776e-5649-4add-b5f9-08baf1d4f43b, 
 
 	ktype_id = dataset-catalog, 
 
 	slug = foo1, 
 
 	annotations = [], 
 
 	attachments = [], 
 
 	linked_kitems = [
 		LinkedKItem(id=7664eeae-b283-40ce-aa26-607b871ccd2a, source_id=d475776e-5649-4add-b5f9-08baf1d4f43b)
 	], 
 
 	affiliations = [], 
 
 	authors = [
 		Author(user_id=4440f8c8-f443-4114-a7a7-d8e2a6b5d776)
 	], 
 
 	avatar_exists = False, 
 
 	contacts = [], 
 
 	created_at = 2024-02-27 15:41:24.671471, 
 
 	updated_at = 2024-02-27 15:41:24.671471, 
 
 	external_links = [], 
 
 	kitem_apps = [], 
 
 	summary = None, 
 
 	user_groups = [], 
 
 	custom_properties = CustomProperties(content={})
 )]

... and for all of type `Organization` with the annotation `www.example.org/foo`:

In [24]:
dsms.search(
        ktypes=[dsms.ktypes.Organization], annotations=["www.example.org/foo"]
    )

[KItem(
 
 	name = foo 2, 
 
 	id = 7664eeae-b283-40ce-aa26-607b871ccd2a, 
 
 	ktype_id = organization, 
 
 	slug = foo2, 
 
 	annotations = [
 		Annotation(iri=www.example.org/foo, name=foo, namespace=www.example.org)
 	], 
 
 	attachments = [], 
 
 	linked_kitems = [
 		LinkedKItem(id=d475776e-5649-4add-b5f9-08baf1d4f43b, source_id=7664eeae-b283-40ce-aa26-607b871ccd2a)
 	], 
 
 	affiliations = [], 
 
 	authors = [
 		Author(user_id=4440f8c8-f443-4114-a7a7-d8e2a6b5d776)
 	], 
 
 	avatar_exists = False, 
 
 	contacts = [], 
 
 	created_at = 2024-02-27 15:41:26.180197, 
 
 	updated_at = 2024-02-27 15:41:26.180197, 
 
 	external_links = [], 
 
 	kitem_apps = [], 
 
 	summary = None, 
 
 	user_groups = [], 
 
 	custom_properties = CustomProperties(content={})
 ),
 KItem(
 
 	name = foo 3, 
 
 	id = ece40fe8-d25d-4dae-ab80-9cfcb9d354c5, 
 
 	ktype_id = organization, 
 
 	slug = foo3, 
 
 	annotations = [], 
 
 	attachments = [], 
 
 	linked_kitems = [], 
 
 	affiliations = [], 
 
 	authors

Clean up the DSMS from the tutortial:

In [25]:
del dsms[item]
del dsms[item2]
del dsms[item3]
del dsms[item4]

dsms.commit()

dsms.kitems

[]

### 6. Apps

We can investigate which apps are available through JupyterLab:

In [26]:
dsms.apps

[App(filename='deepu_test/csv_tensile_test/csv_tensile_test.ipynb', basename='csv_tensile_test.ipynb', folder='deepu_test/csv_tensile_test')]